# Get consecutive pairs from an iterable

Sometimes, if you're handling an indeterminate list of related items, it might make sense to pass them in a single, flat list where one value and the one following it are related.

This is somewhat similar to how the arguments to [pandas.option_context](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.option_context.html) are handled.

You can use the built-in [zip()](https://docs.python.org/3/library/functions.html#zip) function and the stride option to Python's list slicing to iterate over the pairs of consecutive items.

I learned this from [this StackOverflow answer](https://stackoverflow.com/a/2631222).

In [1]:
import pandas as pd

In [2]:
sample_list = [1, 2, 3, 4, 5, 6]

You're probably familiar with Python's list slicing syntax. For example, this grabs the first two items from the list:

In [3]:
sample_list[0:2]

[1, 2]

And this excludes the first item:

In [4]:
sample_list[1:]

[2, 3, 4, 5, 6]

What I didn't know is that you can supply a stride argument to the slice notation that indicates how many indices the slice should move to get the next item. For example, this gets every other item from the list, starting with the first item:

In [5]:
sample_list[0::2]

[1, 3, 5]

And this gets every other item, starting with the second item:

In [6]:
sample_list[1::2]

[2, 4, 6]

We can put this together using the [`zip()`](https://docs.python.org/3/library/functions.html#zip) function, which combines the corresponding items from multiple iterables into tuples:

In [7]:
# This is wrapped in `list()` for display purposes. `zip()` returns an
# iterator, so we can't see the values until we iterate over it.
list(zip(sample_list[0::2], sample_list[1::2]))

[(1, 2), (3, 4), (5, 6)]

Let's look at this in a pandas context. Assume I have data that consists of columns that are consecutive pairs of numerators and denominators and I want to calculate the fractions using these pairs.

Note that in this toy data, the numerator and denominators are all multiples of each other, so the final fractions should be the same for each row.

In [8]:
data = [
    (1, 2, 2, 3, 3, 4),
    (2, 4, 4, 6, 6, 8),
    (4, 8, 8, 12, 12, 16),    
]
columns = [
    'numerator_1',
    'denominator_1',
    'numerator_2',
    'denominator_2',
    'numerator_3',
    'denominator_3',
]

df = pd.DataFrame.from_records(data, columns=columns)

df

,numerator_1,denominator_1,numerator_2,denominator_2,numerator_3,denominator_3
0,1,2,2,3,3,4
1,2,4,4,6,6,8
2,4,8,8,12,12,16


In [9]:
def calc_ratios(row):
    """Calculate ratios in columns that are numerator, denominator pairs"""
    ratios = []
    col_names = []
    # Here's where we use list slicing, with the stride argument and zip()
    for num_col, denom_col in zip(row.index[0::2], row.index[1::2]):
        col_names.append(num_col.replace("numerator", "ratio"))
        ratios.append(row[num_col] / row[denom_col])
        
    return pd.Series(ratios, index=col_names)
    
df.apply(calc_ratios, axis=1)

,ratio_1,ratio_2,ratio_3
0,0.5,0.666667,0.75
1,0.5,0.666667,0.75
2,0.5,0.666667,0.75
